In [1]:
:opt no-lint

# Variables and Conditoinals
(변수와 조건식)

덧셈만 하던 언어에 이름(변수)과 조건식을 추가해 보자.

* 문법 syntax

  $\begin{array}{lll}
    n \in \textsf{Int} & \\
    x \in \textsf{String} & \\
    e \in \textsf{Expr} &::=& n \\ 
                  &\quad\mid& e_1 \;✚\; e_2 \\
                  &\quad\mid& \texttt{if}~ e ~\texttt{then}~ e_1 ~\texttt{else}~ e_0 \\
                  &\quad\mid& x
    \end{array}$

* 실행 환경 execution environment (유한한 개수의 이름을 값으로 대응시키는 함수) 

  $\textsf{Env} ~=~ \textsf{String} \stackrel{\textrm{fin}}{\longrightarrow} \textsf{Int}$

  $\sigma \in \textsf{Env} ~::=~ \{ x_1\mapsto n_1, \cdots, x_k\mapsto n_k\}$
  
  $\mathop{\mathrm{dom}}(\sigma) = \{x_1,\cdots,x_k\}$
  
  $\sigma(x_i) = n_i \qquad(\,x_i\in\mathop{\mathrm{dom}}(\sigma)\,)$

* 계산 규칙 evaluation rules ($\sigma,e \Downarrow n$)

  $\displaystyle
  \frac{~~}{\sigma,n \Downarrow n}
  \qquad
  \frac{\sigma,e_1 \Downarrow n_1 \quad \sigma,e_2 \Downarrow n_2}{
        \sigma,e_1 \,✚\, e_2 \Downarrow n_1+n_2}$
        
  $\displaystyle
  \frac{\sigma,e \Downarrow 0 \quad \sigma,e_0 \Downarrow n_0}{
        \sigma,\texttt{if}~ e ~\texttt{then}~ e_1 ~\texttt{else}~ e_0 \Downarrow n_0}
  \qquad        
  \frac{\sigma,e \Downarrow n \quad \sigma,e_1 \Downarrow n_1}{
        \sigma,\texttt{if}~ e ~\texttt{then}~ e_1 ~\texttt{else}~ e_0 \Downarrow n_1}~(n\neq 0)$
  
  $\displaystyle
  \frac{~~}{\sigma,x \Downarrow \sigma(x)}
  $

In [2]:
data Expr = Val Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          | Var String         -- x
          deriving (Eq,Ord,Show)

-- 실행시 이름에 대응되는 값을 찾아볼 수 있는 (실행)환경(environment)
type Env = [ (String, Int) ]  

eval :: Env -> Expr -> Int
eval env (Val n) = n
eval env (Add e1 e2) = eval env e1 + eval env e2
eval env (If e e1 e0) = if eval env e == 0
                           then eval env e0
                           else eval env e1
                    {-- case eval env e of
                          0 -> eval env e0
                          _ -> eval env e1 -}
eval env (Var x) = case lookup x env of
                     Nothing -> error (x ++ " not found") 
                     Just n  -> n

`lookup`이라는 함수는 찾으면 데이터 생성자 `Just`로 포장된 값을, 없으면 데이터 상수 `Nothing`을 돌려준다.
참고로 `Just`와 `Nothing`은 아래와 같이 표준라이브러리에 정의되어 있는 `Maybe` 타입의 데이타 생성자 및 상수이다.
```haskell
data Maybe a = Nothing | Just a
```

In [3]:
:type lookup
lookup "x" [("x",3),("y",4)]
lookup "y" [("x",3),("y",4)]
lookup "z" [("x",3),("y",4)]

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

Just 3

Just 4

Nothing

`if b then (x + 3) else y`와 같은 코드를 `Expr` 타입의 하스켈 데이터로 다음과 같이 나타낼 수 있다.

In [4]:
e = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")

이 코드를 실행하려면 $\sigma = \{ x \mapsto 2, y \mapsto 4, b \mapsto 0 \}$과 같은 환경이 필요하다.
인터프리터 함수에 환경과 식을 같이 넘겨 ($\texttt{eval}~\,\sigma\,~\texttt{e}$) 실행하면 된다. 

In [5]:
eval [("x",2), ("y",4), ("b",0)] e

4

이번에는 $\{ x \mapsto 2, y \mapsto 4, b \mapsto 1 \}$ 환경으로 실행해 보자.

In [6]:
eval [("x",2), ("y",4), ("b",1)] e  

5

실행 환경은 코드에 나타나는 이름에 대한 값을 모두 포함하고 있어야 한다.
그렇지 않을 경우 예컨대 $\{ x \mapsto 2, y \mapsto 4 \}$ 같은 환경으로 실행한다면 ...

In [7]:
eval [("x",2), ("y",4)]          e -- 에러 발생

: 

----
연습문제 04-01

````if x then y else (if y then y+z else z)````
에 해당하는 `Expr` 타입의 식 `e4`를 정의하라.

그리고 다음 세 가지 실행환경을 정의하여
각각의 환경에서 `eval`함수로 `e4`를 실행해 보라.

* $\sigma_1 = \{x\mapsto 1, y\mapsto 2, z\mapsto 3\}$
* $\sigma_3 = \{x\mapsto 1, y\mapsto 0, z\mapsto 3\}$
* $\sigma_2 = \{x\mapsto 0, y\mapsto 2, z\mapsto 3\}$

In [8]:
e4 = undefined -- 여기에 작성

sigma1 = undefined -- 여기에 작성
sigma2 = undefined -- 여기에 작성
sigma3 = undefined -- 여기에 작성

-- eval 함수로 각각의 환경에서 e4를 실행해 어떤 결과값이 나오나 확인해보기

----
### 식의 간소화(simplification) 규칙

여러 번 실행해야 하는 프로그램이라면 실행 전에 간소화할 수 있는 부분은 간소화해 놓는 것이 효율적이다.
```
0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + x + y
```
위와 같은 프로그램은 `x + y`로 간소화해 놓으면 매번 실행할 때마다 쓸데없이 0을 여러 번 덧셈하지 않아도 된다.

이런 것을 엄밀하게 규칙을 정해 표현해 보자.
이를테면 아래와 같은 간소화 규칙을 따라 간소화하자는 것이다.

$\displaystyle
 \frac{~~}{n_1 \,✚\, n_2 ~\rightsquigarrow~ n_1+n_2}
 \qquad
 \frac{~~}{e_1 \,✚\, 0 ~\rightsquigarrow~ e_1}
 \qquad
 \frac{~~}{0 \,✚\, e_2 ~\rightsquigarrow~ e_2}
$

$\displaystyle
 \frac{~~}{\textbf{if}~0~\textbf{then}~e_1~\textbf{else}~e_0 ~\rightsquigarrow~ e_0}
 \qquad
 \frac{~~}{\textbf{if}~n~\textbf{then}~e_1~\textbf{else}~e_0 ~\rightsquigarrow~ e_1} ~ (n\neq 0)
$

위의 규칙은 전체 식이 저렇게 생겼을 때 적용할 수 있는 규칙이다.
그래서 위와 같은 규칙만 있다면<br>
`if 3+4 then 0+x else y+0`이나<br>
`x + (if 0 then y else z)` 같이<br>
더 큰 식 안에 줄일 수 있는 부분식이 들어 있는 경우를 처리하지 못한다.

주어진 전체 식이 어떤 큰 식 $\mathcal{C}[e]$일 때
그 일부분을 이루고 있는 부분식 $e$를 $e'$ 간소화하는 것이 가능하다면,
바로 그 부분만 $e'$으로 바꿔친 $\mathcal{C}[e']$로 전체 식을 간소화할 수 있어야
바로 위에서 예로 든 식들을 충분히 간소화할 수 있을 것이다.

예를 들어  $\mathcal{C} = (1 + [\,\cdot\,]) + 3$ 일 때,
$\mathcal{C}[e] = (1 + e) + 3$ 이 된다.

바로 아래의 규칙이 문맥(context)라는 개념을 이용해 이렇게 큰 식의 일부분을 이루는 
부분식(subexpression)을 간소화한다는 생각을 엄밀하게 기술하고 있다.

$\displaystyle
 \frac{e ~\rightsquigarrow~ e'}{\mathcal{C}[\,e\,]  ~\rightsquigarrow~ \mathcal{C}[\,e']} ~~ (\mathcal{C}\neq [\,\cdot\,])
$

context for simplification 간소화를 위한 문맥

$\displaystyle
\begin{array}{lrl}
 \mathcal{C} \in \textsf{Ctx} &::=& [\,\cdot\,] \\
            &\mid& \mathcal{C}\,✚\,e_2 \\
            &\mid& e_1\,✚\,\mathcal{C} \\
            &\mid& \textbf{if}~\mathcal{C}~\textbf{then}~e_1~\textbf{else}~e_0 \\
            &\mid& \textbf{if}~e~\textbf{then}~\mathcal{C}~\textbf{else}~e_0 \\
            &\mid& \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~\mathcal{C}
\end{array}$

$\mathcal{C}$라는 문맥(context)은 구멍이 하나 뚫려 있는 식으로 이해할 수 있으며,
위의 정의는 구멍이 구체적으로 어디에 뚫려 있는 것이 가능한지에 대한 재귀적 정의이다.
그리고 $\mathcal{C}[\,e\,]$는 바로 그 구멍을 $e$를 부분식으로 메워넣은 식을 뜻한다.

이런 문맥(context)라는 개념을 활용하는 이유는 식의 종류(덧셈식, 조건식, ...)가 많아짐에 따라
부분식을 처리하기 규칙의 개수도 같이 늘어나기 때문에 부분식을 찾아들어가는 규칙을 문맥을 이용한
하나의 규칙으로 요약하기 위함이다. 만일 문맥 $\mathcal{C}$를 사용하지 않고 풀어서 나열한다면 다음과
같이 여러개의 규칙으로 부분식을 간소화하는 방법을 나타내야 할 것이다.

$\displaystyle
 \frac{e_1 ~\rightsquigarrow~ e_1'}{e_1 \,✚\, e_2 ~\rightsquigarrow~ e_1' \,✚\, e_2}
 \qquad
 \frac{e_2 ~\rightsquigarrow~ e_2'}{e_1 \,✚\, e_2 ~\rightsquigarrow~ e_1 \,✚\, e_2'}
$

$\displaystyle
 \frac{e ~\rightsquigarrow~ e'}{\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0 ~\rightsquigarrow~ \textbf{if}~e'~\textbf{then}~e_1~\textbf{else}~e_0}
$

$\displaystyle
 \frac{e_1 ~\rightsquigarrow~ e_1'}{\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0 ~\rightsquigarrow~ \textbf{if}~e~\textbf{then}~e_1'~\textbf{else}~e_0}
$

$\displaystyle
 \frac{e_0 ~\rightsquigarrow~ e_0'}{\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0 ~\rightsquigarrow~ \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0'}
$

In [9]:
-- 부분식을 줄이는 문맥 활용 규칙을 제외한 처음 5가지 규칙에 따라
-- 식(Expr)을 한 단계 간소화하는 simp 함수를 작성해 보자.
-- 단, 적용할 규칙이 없는 식은 원래 식 그대로를 돌려주기로 한다.

simp :: Expr -> Expr
simp (Val n1 `Add` Val n2) = Val (n1+n2)
simp (Val 0 `Add` e2) = e2
simp (e1 `Add` Val 0) = e1
-- simp (If ...) = ... -- if 0 e1 e0 규칙
-- simp (If ...) = ... -- if n e1 e0 규칙 (n/=0일 때)
simp e = e -- 해당하는 규칙이 없으면 원래 식을 그대로 결과로

In [10]:
-- 0+(x+0) ~~> x+0 ~~> x 를 simp를 이용해 실행해 확인해 보기 
e5 = Val 0 `Add` (Var "x" `Add` Val 0)

e5                      -- simp 0 번 적용해 0+(x+0)
simp e5                 -- simp 1 번 적용해    x+0
simp . simp $ e5        -- simp 2 번 적용      x
simp . simp . simp $ e5 -- simp 3 번 적용      x    (더 이상 간소화할 것이 없어 이전과 동일한 결과)

Add (Val 0) (Add (Var "x") (Val 0))

Add (Var "x") (Val 0)

Var "x"

Var "x"

---
연습문제 04-02 

`If`에 대한 간소화 규칙 둘을 각각 simp의 등식으로 추가로 작성하여 완성하고<br>
아래에 `if 0 then x else (if 1 then y else z)`에 해당하는 `e6`를 정의한 후<br>
simp를 반복적으로 적용해 아래와 같이 간소화가 일어나는지 확인해 보라.<br>
(위에서 덧셈식 `e5`에 대해 실행했던 것처럼 해보라 이야기다.)
````
if 0 then x else (if 1 then y else z)
              ~~> if 1 then y else z
                        ~~> y
````

In [11]:
e6 = undefined

-- simp를 e6에 여러 번 적용해 결과를 확인해 보라

----
# 과제 HW2 (총 11점)

HW2과제 ??그룹
* 이름: ???
* 이름: ???
* 이름: ???

(1)번과 (2)번 각각의 문제마다 반드시 2개 이상의 테스트를 작성해야만 한다. (그렇지 않으면 함수를 맞게 작성했더라도 점수는 0점으로 처리)

마지막 3번의 경우는 사실상 2개의 테스트가 이미 주어져 있는 것이나 마찬가지이므로 그 2개를 테스트해 보고 추가로 테스트 하나만 더 작성하라. (마찬가지로 이렇게 테스트를 작성하지 않으면 0점 처리)

## (1) `dom :: Env -> [String]` (2점)

저 앞에서 실행 환경을 소개할 때 정의역 즉 실행 환경에 등록된 변수들을 뽑아내는 dom이라는 함수도 함께 정의했다.

이를 하스켈 함수 `dom :: Env -> [String]`으로 작성하라.

예컨대,
````haskell
dom [("x",3),("y",-2),("z",4)] == ["x","y","z"]
````

In [12]:
-- dom :: Env -> [String]

## (2) `fv :: Expr -> [String]` (3점)

산술식(`Expr`)에 나타나는 변수들의 리스트를 계산하는 함수 fv를 다음과 같이 정의할 수 있다.

$\displaystyle
\begin{array}{lll}
 \mathop{\textrm{fv}}(\,n\,) &=& \{\;\} \\
 \mathop{\textrm{fv}}(\,e_1 \;✚\; e_2\,) &=&
    \mathop{\textrm{fv}}(e_1) \;\cup\; \mathop{\textrm{fv}}(e_2) \\
 \mathop{\textrm{fv}}(\,\texttt{if}~e~\texttt{then}~e_1~\texttt{else}~e_0\,) &=&
    \mathop{\textrm{fv}}(e_0) \;\cup\; \mathop{\textrm{fv}}(e_1) \;\cup\; \mathop{\textrm{fv}}(e_0)  \\
 \mathop{\textrm{fv}}(\,x\,) &=& \{x\}
\end{array}$

예컨대, $\mathop{\textrm{fv}}(\,\texttt{if}~3+x~\texttt{then}~x+y~\texttt{else}~z+x\,) = \{x,y,z\}$

이를 하스켈 함수  `fv :: Expr -> [String]`로 작성하라. 그리고 `fv`를 사용하는 테스트도 4개 이상 이상 작성해 보라.

In [13]:
import Data.List (union) -- 리스트를 집합으로 생각해 합집합 연산을 하는 union 함수 불러오기

union ["x","y","z"] ["y","w"]    -- 기본적인 함수로 호출 형태로 활용

["x","y","z"] `union` ["y","w"]  -- 역따옴표로 중위(infix) 연산자처럼 활용

["x","y","z","w"]

["x","y","z","w"]

In [14]:
import Data.List (union)

-- fv :: Expr -> [String]

-- fv를 사용하는 테스트도 네 개 이상 작성해 보라

## (3) `simplify :: Expr -> Expr` (3점)

위의 연습문제에서는 `simp` 함수를 반복적으로 여러 번 수작업으로 코드를 여러 번 반복하며 적용해 보았다.

이를 자동으로 수행하는 재귀함수 `simplify` 함수를 작성하라.
`simplify`는 더 이상 적용할 간소화 규칙이 없을 때까지 반복해서 `simp` 함수를 활용하여 식을 간소화하도록 작성하면 된다.
(종료조건이 무엇으로 할지 생각해 보라.)

예컨대,  
````haskell
simplify e5 == Var "x"
simplify e6 == Var "y"
````

참고로 `simp` 함수 정의에서 부분식의 간소화를 위해 문맥(context)를 활용하는 규칙은 구현하지 않았으므로 당연히 `simplify` 함수도 전체 식이 아닌 부분식 안으로 들어가 간소화하는 것은 처리하지 않는다.
이를테면 `(x + 0) + (0 + y)`같은 식은 우리가 눈으로 봐서 금방 간소화할 수 있는 식이지만, 왼쪽 부분식(`x + 0`)이나 오른쪽 부분식(`0 + y`) 안으로 들어가 규칙을 적용하지 않는 한,
지금의 `simp` 함수를 단순히 반복 적용하는 것만으로는 간소화할 수 없는 식이므로 `simplify`도 이런 식은 간소화하지 않는다.
(이 부분은 다음 기회에 ...) 

In [15]:
-- simplify :: Expr -> Expr
--

In [16]:
-- +만 연달아 간소화하는 e5와 if만 연달아 간소화하는 e6외에
-- if와 +를 모두 간소화하게 되는 simplify에 대한 테스트도 작성하라
-- 즉 e5, e6 외에 최소한 하나 더 테스트를 작성하여 총 3개 이상의 테스트를 작성하라

## (4) `contextualize :: Expr -> [(Ctx,Expr)]` (4점)
함수 `contextualize`를 주어진 식(`Expr`)에 대한 가능한 모든 문맥화를 리스트로 나열하도록 정의하자.

$\qquad \texttt{contextualize}~e ~=~
  \big[ (\mathcal{C_1},e_1), \cdots, (\mathcal{C_k},e_k) \big] $

즉, $e = \mathcal{C}_i[e_i]$를 만족하는 가능한 모든 문맥($\mathcal{C}_i$)과
그 문맥의 구멍에 위치하는 부분식 $e_i$의 순서쌍을 계산하는 함수이다.

문맥(`Ctx`)은 구멍에 채워넣을 람다식을 받아 람다식을 계산하는 함수(`Expr -> Expr`)로 표현하기로 한다.

최소한 4개의 테스트를 작성하라. 함수를 맞게 작성하더라도 함수가 올바르게 동작한다는 것을 확인하기 위한 적절한 테스트를 작성하지 않으면 0점이다.

In [31]:
type Ctx = Expr -> Expr

-- 문맥은 이런 식으로 정의해서
ctx1 :: Ctx
ctx1 = \hole -> Add (Var "x") (Add hole (Val 3)) -- x + ([.] + 3)

-- 이런 식으로 활용할 수 있다
ctx1 (Var "[.]") -- 구멍이 어디인지 확인하기 위해 "[.]"이라는 이름의 변수를 끼워넣어 보았다
ctx1 (Var "y")    -- 구멍에 변수 y를 끼워넣어 보았다 
ctx1 (If (Var"x") (Val 4) (Val 5)) -- 구멍에 if x then 4 else 5 를 끼워넣어 보았다

Add (Var "x") (Add (Var "[.]") (Val 3))

Add (Var "x") (Add (Var "y") (Val 3))

Add (Var "x") (Add (If (Var "x") (Val 4) (Val 5)) (Val 3))

In [32]:
contextualize :: Expr -> [(Ctx,Expr)]
contextualize (Val n)      = undefined -- undefined 대신에 적절한 내용을 작성하라
contextualize (Add e1 e2)  = undefined -- undefined 대신에 적절한 내용을 작성하라
contextualize (If e e1 e0) = undefined -- undefined 대신에 적절한 내용을 작성하라
contextualize (Var x)      = undefined -- undefined 대신에 적절한 내용을 작성하라

In [ ]:
-- 테스트를 여기에

In [ ]:
-- 테스트를 여기에

In [ ]:
-- 테스트를 여기에

In [ ]:
-- 테스트를 여기에